In [1]:
# import libaries
import numpy as np
import pandas as pd
import nltk, pprint
import matplotlib.pyplot as plt
import random
import itertools

# supress warnings
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,multilabel_confusion_matrix
from sklearn.utils import class_weight



import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,SimpleRNN,LSTM,RepeatVector
from tensorflow.keras.layers import Dense, Input,Dropout,Bidirectional
from tensorflow.keras.layers import TimeDistributed




In [2]:
input=np.random.randint(0,10,size=(2,4))
print(input)
output=np.sort(input)
print(output)

[[5 1 2 2]
 [8 8 0 3]]
[[1 2 2 5]
 [0 3 8 8]]


In [3]:
X=np.random.randint(0,10,size=(10000,100))
y=np.sort(X)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [5]:
len(X_train),len(X_test)

(7000, 3000)

In [6]:
X_train=to_categorical(X_train)
X_test=to_categorical(X_test)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [7]:
X_train.shape,y_train.shape

((7000, 100, 10), (7000, 100, 10))

## Many to Many

In [8]:
model=Sequential()
model.add(Bidirectional(LSTM(64,return_sequences=True),input_shape=(100,10)))
model.add(Bidirectional(LSTM(64,return_sequences=True)))
model.add(Dense(10, activation='softmax'))

In [9]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional (Bidirectional (None, 100, 128)          38400     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
dense (Dense)                (None, 100, 10)           1290      
Total params: 138,506
Trainable params: 138,506
Non-trainable params: 0
_________________________________________________________________


In [11]:
model.fit(X_train,y_train, batch_size=32,epochs=10)

Epoch 1/10
219/219 [==============================] - 15s 17ms/step - loss: 1.2083 - acc: 0.5309
Epoch 2/10
219/219 [==============================] - 4s 16ms/step - loss: 0.5838 - acc: 0.7437
Epoch 3/10
219/219 [==============================] - 4s 16ms/step - loss: 0.4099 - acc: 0.8231
Epoch 4/10
219/219 [==============================] - 4s 17ms/step - loss: 0.3210 - acc: 0.8611
Epoch 5/10
219/219 [==============================] - 4s 16ms/step - loss: 0.2805 - acc: 0.8791
Epoch 6/10
219/219 [==============================] - 4s 17ms/step - loss: 0.2418 - acc: 0.8956
Epoch 7/10
219/219 [==============================] - 4s 16ms/step - loss: 0.2058 - acc: 0.9118
Epoch 8/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1833 - acc: 0.9214
Epoch 9/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1569 - acc: 0.9329
Epoch 10/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1227 - acc: 0.9504


In [12]:
model.evaluate(X_test,y_test)

94/94 [==============================] - 2s 8ms/step - loss: 0.1239 - acc: 0.9512


[0.12390192598104477, 0.951170027256012]

In [13]:
pred=model.predict(X_test)

In [14]:
# X_test[0].shape
rand=np.random.randint(len(X_test))
print(rand)
print([range(0,10)[np.argmax(i)] for i in X_test[rand]])
print([range(0,10)[np.argmax(i)] for i in y_test[rand]])
print("-"*100)
print([range(0,10)[np.argmax(i)] for i in pred[rand]])


8
[9, 0, 3, 2, 5, 4, 5, 7, 2, 8, 8, 3, 6, 3, 4, 8, 1, 2, 2, 3, 2, 1, 2, 2, 5, 2, 5, 5, 0, 5, 4, 8, 5, 0, 6, 1, 6, 0, 1, 7, 3, 3, 1, 5, 7, 0, 6, 8, 5, 8, 9, 8, 9, 0, 3, 9, 6, 7, 4, 5, 7, 7, 0, 1, 8, 4, 6, 3, 0, 3, 0, 5, 3, 8, 8, 5, 5, 4, 1, 0, 9, 5, 0, 2, 5, 9, 5, 3, 8, 3, 9, 0, 6, 2, 6, 2, 9, 5, 0, 4]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
----------------------------------------------------------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 

## Seq2Seq

In [15]:
# instantiate model
model = Sequential()

# encoder with multiple inputs	
# return_sequences =False focus only after the last time step
model.add(Bidirectional(LSTM(64), input_shape=(100,10)))
# here we do not return_sequence
# if we do not use RepeatVector the output dimentions of the encoder will be 2d (None,128) but the decoder will expect (None,100,128)
# and the meaning of it will be learnt by the model during back propagatioin

# encoded sequence	
model.add(RepeatVector(100))
		
model.add(Bidirectional(LSTM(64, return_sequences=True)))

# TimeDistributed(): focus on every time step
model.add(TimeDistributed(Dense(10, activation='softmax')))

In [16]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['acc'])

In [17]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_2 (Bidirection (None, 128)               38400     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 100, 128)          0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 100, 128)          98816     
_________________________________________________________________
time_distributed (TimeDistri (None, 100, 10)           1290      
Total params: 138,506
Trainable params: 138,506
Non-trainable params: 0
_________________________________________________________________


In [18]:
model.fit(X_train,y_train, batch_size=32,epochs=10)

Epoch 1/10
219/219 [==============================] - 8s 17ms/step - loss: 1.3558 - acc: 0.5116
Epoch 2/10
219/219 [==============================] - 4s 16ms/step - loss: 0.3746 - acc: 0.8489
Epoch 3/10
219/219 [==============================] - 4s 16ms/step - loss: 0.2731 - acc: 0.8858
Epoch 4/10
219/219 [==============================] - 4s 16ms/step - loss: 0.2209 - acc: 0.9078
Epoch 5/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1891 - acc: 0.9208
Epoch 6/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1617 - acc: 0.9322
Epoch 7/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1320 - acc: 0.9460
Epoch 8/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1162 - acc: 0.9521
Epoch 9/10
219/219 [==============================] - 4s 16ms/step - loss: 0.1030 - acc: 0.9569
Epoch 10/10
219/219 [==============================] - 4s 16ms/step - loss: 0.0773 - acc: 0.9708


In [19]:
model.evaluate(X_test,y_test)

94/94 [==============================] - 2s 8ms/step - loss: 0.0846 - acc: 0.9654


[0.08462662249803543, 0.9653566479682922]

In [20]:
pred=model.predict(X_test)
rand=np.random.randint(len(X_test))
print(rand)
print([range(0,10)[np.argmax(i)] for i in X_test[rand]])
print([range(0,10)[np.argmax(i)] for i in y_test[rand]])
print("-"*100)
print([range(0,10)[np.argmax(i)] for i in pred[rand]])

1112
[2, 9, 2, 5, 5, 5, 5, 6, 7, 3, 1, 0, 8, 8, 8, 4, 7, 9, 8, 4, 0, 5, 4, 6, 2, 3, 6, 0, 3, 2, 9, 4, 3, 3, 4, 2, 2, 5, 8, 0, 5, 6, 7, 2, 0, 6, 7, 9, 1, 4, 3, 5, 4, 1, 0, 3, 3, 8, 3, 4, 6, 1, 7, 8, 8, 2, 4, 2, 1, 7, 7, 8, 3, 2, 1, 1, 9, 3, 0, 7, 2, 3, 0, 3, 0, 7, 6, 5, 8, 9, 5, 3, 2, 2, 8, 0, 6, 0, 3, 6]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9]
----------------------------------------------------------------------------------------------------
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 